In [1]:
dayObs = '20251026'
run = "None"
sensor='R10_S10'
guidersensor='R44_SG0'

In [2]:
from lsst.daf.butler import Butler
from astropy.table import Table
from astropy.time import Time, TimeDelta
import tabulate
import numpy

In [3]:
if dayObs=='Today':
    dayObs = (Time.now() - TimeDelta(0.5, format='jd')).iso[:10].replace('-','')

In [4]:
butler = Butler('embargo_new', collections=[
    'LSSTCam/raw/all',
#    'LSSTCam/calib/unbounded',
#    'LSSTCam/photodiode'
])

In [5]:
exposureList = []

if run == "None":
    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"
else:
    where=f"exposure.science_program='{run}' and instrument='LSSTCam'"    

for record in butler.registry.queryDimensionRecords("exposure", where=where):
    nimage=len(list(butler.registry.queryDatasets('raw',dataId=record.dataId)))
    guider=len(list(butler.registry.queryDatasets('photodiode',dataId=record.dataId)))
    exposureList.append([record.id, record, nimage, guider])
exposureList.sort(key=lambda x: x[0])
table = Table(rows=[ (
            f'<a href=http://lsstcam-mcm.cp.lsst.org/FITSInfo/view.html?image={record.obs_id} target="_blank">{record.obs_id}</a>',
            record.timespan.begin.iso,
            record.seq_num,
            record.observation_type,
            record.science_program,
            record.observation_reason,
            record.exposure_time,
            record.physical_filter,
            nimage,
            f'<a href=https://usdf-rsp.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/GuiderData?expid={record.obs_id}&sensor={guidersensor}&ts_hide_code=1 target="_blank">Guider</a>',
            f'<a href=https://usdf-rsp.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/FitsHeader?expid={record.obs_id}&detector={sensor} target="_blank" >meta</a>',    
                     )
            for [id,record,nimage, guider] in exposureList ]
            )

In [6]:
subtable=Table(rows=[  ( record.obs_id, record.seq_num, nimage ) for [id,record,nimage, guider] in exposureList ], names=(  'id', 'seqnum', 'nimage'  ) )

# Possible data transfer issue

In [7]:
for a, b in zip(subtable[:-1][numpy.diff(subtable['seqnum'])!=1], subtable[1:][numpy.diff(subtable['seqnum'])!=1]):
    print(a['id'], b['id'])

# Changes in number of images ingested

In [8]:
for a, b in zip(subtable[:-1][numpy.diff(subtable['nimage'])!=0], subtable[1:][numpy.diff(subtable['nimage'])!=0]):
    print(a['id'], a['nimage'], b['id'], b['nimage'])

# Full records

In [9]:
print(f"The number of exposures is {len(table)}")
tabulate.tabulate(table[::-1], tablefmt='unsafehtml',headers=[
                   'obs_id', 'time', 'seqnum', 'observation_type',
                   'science_program', 'observation_reason','exposure_time',
                   'physical_filter', 'nimage', 'Guider', 'metadata'
                  ])

The number of exposures is 23


obs_id,time,seqnum,observation_type,science_program,observation_reason,exposure_time,physical_filter,nimage,Guider,metadata
MC_O_20251026_000023,2025-10-26 19:33:01.454055,23,dark,unknown,test,15,none,197,Guider,meta
MC_O_20251026_000022,2025-10-26 19:32:42.868881,22,dark,unknown,test,15,none,197,Guider,meta
MC_O_20251026_000021,2025-10-26 19:32:24.279644,21,dark,unknown,test,15,none,197,Guider,meta
MC_O_20251026_000020,2025-10-26 19:32:05.812273,20,dark,unknown,test,15,none,197,Guider,meta
MC_O_20251026_000019,2025-10-26 19:31:47.200867,19,dark,unknown,test,15,none,197,Guider,meta
MC_O_20251026_000018,2025-10-26 19:31:28.547699,18,dark,unknown,test,15,none,197,Guider,meta
MC_O_20251026_000017,2025-10-26 19:31:09.918872,17,dark,unknown,test,15,none,197,Guider,meta
MC_O_20251026_000016,2025-10-26 19:30:51.711327,16,dark,unknown,test,15,none,197,Guider,meta
MC_O_20251026_000015,2025-10-26 19:30:33.304265,15,dark,unknown,test,15,none,197,Guider,meta
MC_O_20251026_000014,2025-10-26 19:30:14.845785,14,dark,unknown,test,15,none,197,Guider,meta
